## Transfer learning to recognise new pictures

In [1]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [6]:
import argparse 

args = argparse.Namespace()
args.train_full_folder = './data/transfer-pic/'
args.train_full_car_folder = args.train_full_folder + 'car/'
args.train_full_no_car_folder = args.train_full_car_folder + 'no-car/'
args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 1

In [7]:

def preprocess_images():
    dataset = tf.keras.preprocessing.image_dataset_from_directory(args.train_full_folder)
    for d in dataset:
        print(d)
        break
#     images = []
#     labels = []
#     for file in os.listdir(args.train_full_car_folder):
#         filename = os.fsdecode(file)
#         if filename.endswith(".jpg"): 
#             filepath = os.path.join(args.train_full_car_folder, filename)

def build_model():
        base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
        avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
        output = keras.layers.Dense(10, activation='softmax')(avg)
        model = keras.Model(inputs=base_model.input, outputs=output)
        return base_model, model
    
    
def train(base_model, model, train_data, val_data, train_base=False, epochs=args.epochs, lr=0.2):
        for l in base_model.layers:
            l.tranable = False
        optimizer = keras.optimizers.SGD(lr=lr, momentum=0.9, decay=0.01)
        model.compile(optimizer=optimizer, loss='crossentropy', metrics=['accuracy'])
        print(model.summary())
        history = model.fit(train_data, epochs=epochs, validation_data=val_data)
        plot_history(history)
        

def pipeline():
    images, labels = preprocess_images()
#     train_idx = len(images) - int(len(images) * args.val_fraction)
#     val_idx = train_idx
    base_model, model = build_model()
    train(base_model, model, images, labels)
    train(base_model, model, images, labels, train_base=True, lr=0.005)
    
preprocess_images()

Found 84 files belonging to 2 classes.
(<tf.Tensor: shape=(32, 256, 256, 3), dtype=float32, numpy=
array([[[[254.       , 254.       , 254.       ],
         [251.       , 255.       , 254.       ],
         [249.       , 255.       , 253.       ],
         ...,
         [ 80.       ,  70.       ,  21.       ],
         [110.875    , 105.       ,  53.375    ],
         [158.       , 151.       ,  83.       ]],

        [[177.875    , 188.375    , 181.125    ],
         [164.625    , 176.5      , 162.875    ],
         [160.375    , 171.25     , 151.       ],
         ...,
         [ 79.625    ,  69.375    ,  22.625    ],
         [151.875    , 145.625    ,  75.375    ],
         [161.75     , 152.75     ,  83.75     ]],

        [[142.5      , 138.5      , 126.5      ],
         [220.875    , 212.       , 184.25     ],
         [252.375    , 247.875    , 220.625    ],
         ...,
         [141.25     , 123.25     ,  47.25     ],
         [129.125    , 117.125    ,  43.125    ],
     